In [72]:
import pandas as pd
import re
from nltk import word_tokenize
from nltk.classify import NaiveBayesClassifier
import nltk.classify.util
from nltk.corpus import stopwords

In [78]:
df_reviews = pd.read_csv('amazon-cell-phones-reviews/20190928-reviews.csv')
df_reviews.loc[df_reviews['rating'] <= 3, 'liked'] = False
df_reviews.loc[df_reviews['rating'] > 3, 'liked'] = True
stop_words = set(stopwords.words('english'))


In [11]:
df_reviews.shape

(82815, 9)

In [12]:
threshold_factor = 0.7
model_index = int(threshold_factor * len(df_reviews))
print(f'model_index:{model_index}')
df_model_data = df_reviews.iloc[:model_index]

model_index:57970
             asin              name  rating               date  verified  \
0      B0000SX2UC             Janet       3   October 11, 2005     False   
1      B0000SX2UC        Luke Wyatt       1    January 7, 2004     False   
2      B0000SX2UC            Brooke       5  December 30, 2003     False   
3      B0000SX2UC     amy m. teague       3     March 18, 2004     False   
4      B0000SX2UC     tristazbimmer       4    August 28, 2005     False   
...           ...               ...     ...                ...       ...   
57965  B077596D7L  Yuliya Moiseyeva       1     August 9, 2019      True   
57966  B077596D7L         Gallagher       1  September 3, 2019      True   
57967  B077596D7L         Trayshawn       4       July 8, 2019      True   
57968  B077596D7L   Amazon Customer       5   January 28, 2019      True   
57969  B077596D7L       yanal matar       4      June 18, 2019      True   

                                              title  \
0             

In [60]:
threshold_factor = 0.8

positive_reviews = df_model_data[df_model_data['liked'] == True]
negative_reviews = df_model_data[df_model_data['liked'] == False]


In [101]:
m_str = 'this is a sentence.and this other 12.90'
re.sub("[^\w]", " ",  m_str).split()
def get_list_words(reviews_str):
    token = str(reviews_str)
    return re.sub("[^\w]", " ",  token).split()

def extract_features(word_list):
    return dict([(word, True) for word in word_list if word.lower() not in stop_words])
#if word not in stop_words

In [102]:
positive_series = [get_list_words(review) for review in positive_reviews['body'].values]
negative_series = [get_list_words(review) for review in negative_reviews['body'].values]

In [105]:
positive_features = [(extract_features(a_review), 'Positive') for a_review in positive_series]
negative_features = [(extract_features(a_review), 'Negative') for a_review in negative_series]

In [106]:
print(len(positive_features))
print(len(negative_features))

36984
20986


In [107]:
# Split the data into train and test (80/20)
threshold_factor = 0.8
threshold_positive = int(threshold_factor * len(positive_features))
threshold_negative = int(threshold_factor * len(negative_features))

In [108]:
features_train = positive_features[:threshold_positive] + negative_features[:threshold_negative]
features_test = positive_features[threshold_positive:] + negative_features[threshold_negative:]  
print("\nNumber of training datapoints:", len(features_train))
print("Number of test datapoints:", len(features_test))


Number of training datapoints: 46375
Number of test datapoints: 11595


In [109]:
# Train a Naive Bayes classifier
classifier = NaiveBayesClassifier.train(features_train)
print("\nAccuracy of the classifier:", nltk.classify.util.accuracy(classifier, features_test))


Accuracy of the classifier: 0.8146614920224234


In [110]:
classifier.show_most_informative_features(15)

Most Informative Features
                Horrible = True           Negati : Positi =     88.7 : 1.0
                   Waste = True           Negati : Positi =     74.6 : 1.0
            Disappointed = True           Negati : Positi =     66.4 : 1.0
                   Worst = True           Negati : Positi =     65.2 : 1.0
               Excelente = True           Positi : Negati =     57.7 : 1.0
               Returning = True           Negati : Positi =     55.8 : 1.0
                Terrible = True           Negati : Positi =     51.1 : 1.0
                  Return = True           Negati : Positi =     45.2 : 1.0
                  LOCKED = True           Negati : Positi =     42.9 : 1.0
                 Perfect = True           Positi : Negati =     35.9 : 1.0
                   Avoid = True           Negati : Positi =     35.8 : 1.0
                  BEWARE = True           Negati : Positi =     32.5 : 1.0
             paperweight = True           Negati : Positi =     26.9 : 1.0